In [17]:
!pip install pytesseract


In [18]:
!apt-get install -y tesseract-ocr
!pip install pytesseract pandas openpyxl xlsxwriter


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [19]:
import cv2
import os
import re
import pytesseract
import requests
import pandas as pd
from PIL import Image
from google.colab import drive
from google.colab import files

In [20]:
# ✅ Step 1: Mount Google Drive
drive.mount('/content/drive')

# Path to your invoice folder
invoice_folder = "/content/drive/MyDrive/invoices"
output_excel = "/content/invoice_data.xlsx"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
# List to store extracted data
invoice_data = []

# Function to preprocess images
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
    image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]  # Binarization
    image = cv2.GaussianBlur(image, (3, 3), 0)  # Reduce noise
    return image

In [22]:
# Process each invoice
i=0
for filename in os.listdir(invoice_folder):
    if filename.endswith(".png"):  # Process only PNG files
        file_path = os.path.join(invoice_folder, filename)

        # Preprocess image
        processed_image = preprocess_image(file_path)

        # Save the processed image (optional, for debugging)
        processed_path = f"/content/processed_{filename}"
        cv2.imwrite(processed_path, processed_image)

        # Extract text using Tesseract
        custom_config = r'--oem 3 --psm 6'  # Assume a single block of text
        text = pytesseract.image_to_string(Image.open(processed_path), config=custom_config)

        # Append to list
        invoice_data.append({"filename": filename, "extracted_text": text})
        print(f"Processed\t{i+1}: {filename}")
        i=i+1

Processed	1: 008.png
Processed	2: 017.png
Processed	3: 010.png
Processed	4: 021.png
Processed	5: 012.png
Processed	6: 009.png
Processed	7: 002.png
Processed	8: 006.png
Processed	9: 020.png
Processed	10: 027.png
Processed	11: 015.png
Processed	12: 025.png
Processed	13: 022.png
Processed	14: 011.png
Processed	15: 003.png
Processed	16: 024.png
Processed	17: 013.png
Processed	18: 007.png
Processed	19: 016.png
Processed	20: 026.png
Processed	21: 029.png
Processed	22: 028.png
Processed	23: 001.png
Processed	24: 023.png
Processed	25: 005.png
Processed	26: 004.png
Processed	27: 019.png
Processed	28: 014.png
Processed	29: 018.png
Processed	30: 042.png
Processed	31: 031.png
Processed	32: 091.png
Processed	33: 055.png
Processed	34: 045.png
Processed	35: 051.png
Processed	36: 084.png
Processed	37: 077.png
Processed	38: 032.png
Processed	39: 033.png
Processed	40: 064.png
Processed	41: 099.png
Processed	42: 094.png
Processed	43: 062.png
Processed	44: 038.png
Processed	45: 086.png
Processed	46: 043.p

In [23]:
# ✅ Save to Excel
# Convert to Pandas DataFrame
df = pd.DataFrame(invoice_data)

# ✅ Save to Excel
df.to_excel(output_excel, index=False)
print(f"Invoice data saved to {output_excel}")


Invoice data saved to /content/invoice_data.xlsx


In [11]:
df.head()


,filename,extracted_text
0,008.png,Johnson Group\n529 Dougherty Roads Suite 736\n...
1,017.png,"Cabrera, Hall and Drake\n52868 Hardy Greens\nP..."
2,010.png,Alvarez-Weaver\n283 Sara Causeway Apt. 186\nSt...
3,021.png,"Cuevas, Moreno and Blackburn\n4898 Tina Meadow..."
4,012.png,Barker-Cameron\n885 Mitchell Estates Apt. 410\...


In [12]:
df['extracted_text'][2]

'Alvarez-Weaver\n283 Sara Causeway Apt. 186\nStephenfort, MD 14522\nVAT Number md66110468277\nCox, Franklin and Bishop\nMegan Anderson\n780 Peters Ridges Apt. 439\nPort Thomas, KY 42677\nSaint Kitts and Nevis Invoice# 467719\nDate 23.08.2010\nAmount Due £42.1\nItem Description Unit Cost Quantity Line total\n10 iterate front-end content 57.98 4.35 782.73\n1 utilize wireless portals 22.22 7.05 543.29\n1 iterate next-generation paradigms 96.5 4.49 394.5\nSubtotal 173.64\nDiscount -16.17% -34.77\nTotal £726.37\nPlease pay your invoice within 30 days of receiving it.\nTerms\nThese are our terms and conditions.\nNotes\nThank you for being our customer\n\x0c'

In [24]:

# Initialize empty lists to store invoice and product data
invoice_list = []
product_list = []

# Loop through each row in df
for index, row in df.iterrows():
    text = row['extracted_text']
    filename = row['filename']

    # ✅ Split text by newline
    lines = text.split("\n")

    # ✅ Extract structured fields using new rules
    seller_company = lines[0].strip() if len(lines) > 0 else ""
    seller_address = ", ".join(lines[1:3]) if len(lines) > 2 else ""
    vat_number = re.search(r"VAT Number (\S+)", text)

    buyer_company = lines[4].strip() if len(lines) > 4 else ""
    buyer_name = lines[5].strip() if len(lines) > 5 else ""
    buyer_address = ", ".join(lines[6:8]) if len(lines) > 7 else ""

    # Extract country (from line 8 until "Invoice" appears)
    country_match = re.search(r"(?<=\n)(.*?)(?=\s*Invoice)", text, re.MULTILINE)
    country = country_match.group(1).strip() if country_match else ""

    invoice_number = re.search(r"Invoice# (\d+)", text)
    invoice_date = re.search(r"Date (\d{2}\.\d{2}\.\d{4})", text)
    amount_due = re.search(r"Amount Due £([\d.]+)", text)
    subtotal = re.search(r"Subtotal ([\d.]+)", text)
    discount = re.search(r"Discount [-]?(\d+.\d+%) [-]?([\d.]+)", text)
    total = re.search(r"Total £([\d.]+)", text)

    # ✅ Extract Product Line Items
    product_lines = re.findall(r"(\d+) (.*?) (\d+\.\d+) (\d+\.\d+) (\d+\.\d+)", text)

    # ✅ Convert extracted data to structured dictionary
    invoice_data = {
        "Invoice_Name": filename,
        "Seller_Company": seller_company,
        "Seller_Address": seller_address,
        "VAT_Number": vat_number.group(1) if vat_number else "",
        "Buyer_Company": buyer_company,
        "Buyer_Name": buyer_name,
        "Buyer_Address": buyer_address,
        "Country": country,
        "Invoice_Number": invoice_number.group(1) if invoice_number else "",
        "Invoice_Date": invoice_date.group(1) if invoice_date else "",
        "Amount_Due": amount_due.group(1) if amount_due else "",
        "Subtotal": subtotal.group(1) if subtotal else "",
        "Discount_Percentage": discount.group(1) if discount else "",
        "Discount_Amount": discount.group(2) if discount else "",
        "Total": total.group(1) if total else "",
    }

    # Append invoice data to list
    invoice_list.append(invoice_data)

    # ✅ Convert product lines into structured data
    for prod in product_lines:
        product_list.append({
            "Invoice_Name": filename,
            "Index": prod[0],
            "Description": prod[1],
            "Unit_Cost": prod[2],
            "Quantity": prod[3],
            "Line_Total": prod[4]
        })




In [25]:
# ✅ Convert to Pandas DataFrame
invoice_df = pd.DataFrame(invoice_list)
products_df = pd.DataFrame(product_list)

# ✅ Save both sheets in a single Excel file
file_path = "/content/invoices_and_products.xlsx"
with pd.ExcelWriter(file_path, engine="xlsxwriter") as writer:
    invoice_df.to_excel(writer, sheet_name="Invoices", index=False)
    products_df.to_excel(writer, sheet_name="Products", index=False)

# ✅ Download the file
files.download(file_path)

print("All invoice and product data successfully extracted and saved in a single Excel file!")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

All invoice and product data successfully extracted and saved in a single Excel file!


In [14]:
products_df

,Invoice_Name,Index,Description,Unit_Cost,Quantity,Line_Total
0,008.png,9,engage e-business partnerships,90.26,3.67,380.08
1,008.png,5,deliver killer infrastructures,33.97,9.77,115.2
2,017.png,3,synthesize end-to-end metrics,48.99,8.92,88.13
3,010.png,10,iterate front-end content,57.98,4.35,782.73
4,010.png,1,utilize wireless portals,22.22,7.05,543.29
...,...,...,...,...,...,...
388,044.png,6,benchmark interactive communities,13.16,3.62,331.74
389,044.png,9,enable granular e-tailers,35.49,8.48,291.32
390,044.png,4,strategize innovative methodologies,59.83,5.0,474.04
391,044.png,4,maximize dynamic platforms,72.63,4.96,36.7


In [15]:
invoice_df

,Invoice_Name,Seller_Company,Seller_Address,VAT_Number,Buyer_Company,Buyer_Name,Buyer_Address,Country,Invoice_Number,Invoice_Date,Amount_Due,Subtotal,Discount_Percentage,Discount_Amount,Total
0,008.png,Johnson Group,"529 Dougherty Roads Suite 736, Greenton, PW 51132",AQ20874224195,White-Freeman,Mary Armstrong,"9741 Amanda Avenue, Ethanhaven, SD 75556",Portugal,357712,03.01.1997,81.84,137.89,18.78%,19.47,441.16
1,017.png,"Cabrera, Hall and Drake","52868 Hardy Greens, Port Tonyamouth, WA 84719",qE95355905579,"Cox, Dean and Simmons",Margaret Brown,"2433 Virginia Prairie Apt. 123, Davidton, TX 1...",French Southern Territories,790538,13.10.2016,114.2,61.01,7.67%,34.65,770.85
2,010.png,Alvarez-Weaver,"283 Sara Causeway Apt. 186, Stephenfort, MD 14522",md66110468277,"Cox, Franklin and Bishop",Megan Anderson,"780 Peters Ridges Apt. 439, Port Thomas, KY 42677",Saint Kitts and Nevis,467719,23.08.2010,42.1,173.64,16.17%,34.77,726.37
3,021.png,"Cuevas, Moreno and Blackburn","4898 Tina Meadows, Lake Shawn, DC 81152",1C44769628359,"Pierce, Joseph and Carroll",Charles Bailey,"05362 Eugene Fall Suite 886, Johnsonberg, CA 7...",Costa Rica,728724,29.03.2019,125.33,148.24,7.75%,21.09,412.06
4,012.png,Barker-Cameron,"885 Mitchell Estates Apt. 410, East Jennifermo...",eb70531331181,Harris Ltd,Robert Fox,"54767 Chelsea Squares Apt. 756, West Anthony, ...",Costa Rica,527781,17.06.1993,189.68,165.76,13.68%,30.31,535.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,067.png,Fisher PLC,"PSC 4983, Box 2427, APO AP 24437",Rt75534071561,Cox-Meyer,Joel Bautista,"663 Fletcher Dale, New Scottstad, PW 64729",Congo,954168,14.11.1998,114.47,58.81,6.82%,13.17,380.79
96,050.png,Mclaughlin-Hall,"5328 Evans Mission, Lake Kennethmouth, MO 55594",iR24485243659,Fisher-Le,Hunter Weaver,"464 Farmer Turnpike Suite 327, Cynthiaberg, AK...",Germany,938211,06.08.2004,190.28,122.15,7.44%,25.0,549.88
97,040.png,"Riddle, George and Peterson","673 Thomas Crescent, East Danielmouth, WA 92959",TP97247950630,Madden-Chase,Rhonda Adkins,"PSC 3086, Box 2889, APO AA 64144",Seychelles,722204,05.09.2022,195.43,54.18,5.76%,31.93,537.44
98,047.png,Jones Inc,"Unit 0116 Box 1885, DPO AA 45362",SJ93696306809,Cole-Rowe,April Bennett,"300 Reyes Squares, Melissaborough, MP 46487",Egypt,937933,15.04.1990,188.57,89.98,15.05%,28.01,762.98


In [26]:
# ✅ Load extracted invoice data (from last code update)
excel_path = "/content/invoices_and_products.xlsx"
xls = pd.ExcelFile(excel_path)

# ✅ Read Data from Sheets
invoice_df = pd.read_excel(xls, sheet_name="Invoices")
products_df = pd.read_excel(xls, sheet_name="Products")

# ✅ Create Invoice JSON Data
invoice_data_list = []

for _, row in invoice_df.iterrows():
    invoice_name = row["Invoice_Name"]

    # ✅ Extract relevant fields
    invoice_json = {
        "invoice_number": row["Invoice_Number"],
        "invoice_date": row["Invoice_Date"],
        "buyer": {
            "company_name": row["Buyer_Company"],
            "name": row["Buyer_Name"],
            "address": row["Buyer_Address"],
            "country": row["Country"]
        },
        "seller": {
            "company_name": row["Seller_Company"],
            "address": row["Seller_Address"],
            "vat_no": row["VAT_Number"]
        },
        "payment": {
            "subtotal": row["Subtotal"],
            "discount_percentage": row["Discount_Percentage"],
            "discount_amount": row["Discount_Amount"],
            "total": row["Total"],
            "due": row["Amount_Due"]
        },
        "products": [
            {
                "description": product["Description"],
                "quantity": product["Quantity"],
                "unit_price": product["Unit_Cost"],
                "total_price": product["Line_Total"]
            }
            for _, product in products_df[products_df["Invoice_Name"] == invoice_name].iterrows()
        ]
    }

    invoice_data_list.append(invoice_json)



ERP_API_URL = "https://anilpushpad2544.activehosted.com/api/3/deals"  # ActiveCampaign API
API_KEY = "7abc4fd8090d0bc79ebc3c8e6acd8d0d93be251c929f0989cb59cd11fce4bf77a1414499"

headers = {
    "Content-Type": "application/json",
    "Api-Token": API_KEY
}

response = requests.get(ERP_API_URL, headers=headers)

if response.status_code == 200:
    print("✅ API Connection Successful!")
    print(response.json())  # Print response to verify
else:
    print(f"❌ Error {response.status_code}: {response.text}")


✅ API Connection Successful!
{'deals': [], 'meta': {'currencies': [], 'total': 0}}
